In [61]:
# Convolutional Neural Network
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim

In [27]:
class Net(nn.Module):
  def __init__(self):
    super(Net , self).__init__()
    self.conv1 = nn.Conv2d( 1 , 6 , 5)
    self.conv2 = nn.Conv2d(6 , 16 , 5)
    self.fc1 = nn.Linear(16*5*5 , 120)
    self.fc2 = nn.Linear(120 , 84)
    self.fc3 = nn.Linear(84 , 10)
  def forward(self, x):
    x = f.max_pool2d(f.relu(self.conv1(x)),(2,2))
    x = f.max_pool2d(f.relu(self.conv2(x)),(2,2))
    x = torch.flatten(x,1)
    x = f.relu(self.fc1(x))
    x = f.relu(self.fc2(x))
    x = self.fc3(x)
    return x
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [28]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) #conv1

10
torch.Size([6, 1, 5, 5])


In [36]:
input = torch.randn(1,1,32,32)
output = net(input)
print(output)

tensor([[ 0.1120, -0.0836,  0.0474,  0.0397,  0.0562,  0.0384, -0.0613, -0.0853,
          0.0576, -0.0044]], grad_fn=<AddmmBackward0>)


In [37]:
# Zero the gradient buffers of all parameters and backprops with random gradients:
net.zero_grad()
output.backward(torch.randn(1,10))

In [44]:
# Computing the Loss
target = torch.randn(10)# dummy target
target = target.view(1,-1) # make it the same shape as output
algorithm = nn.MSELoss()
loss = algorithm(output , target)
print(loss)

tensor(1.2107, grad_fn=<MseLossBackward0>)


In [51]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [66]:
#Backprop
#net.zero_grad()
#print('conv1.bias.grad before backward')
#print(net.conv1.bias.grad)
loss.backward()
print('conv1.bias.grad after backward')
print(net.parameters())

RuntimeError: ignored

In [69]:
optimizer = optim.SGD(net.parameters() , lr=0.01)
optimizer.zero_grad() # zero the gradient buffers
output = net(input)
loss = algorithm ( output , target)
loss.backward()
optimizer.step()


tensor(1.1776, grad_fn=<MseLossBackward0>)
